In [33]:
import os

CODEBASE_DIR_NAME = 'oa-small-dev'
GIT_PATH = os.path.dirname(os.getcwd())
CODEBASE_ROOT = os.path.join(GIT_PATH, CODEBASE_DIR_NAME)
CODEBASE_ROOT

'/Users/macbook/Documents/GitHub/oa-small-dev'

In [48]:
import os
import pickle

class File:
    def __init__(self, name, size):
        self.name = name
        self.size = size  # in bytes

    def __repr__(self):
        return f"File(name={self.name}, size={self.size} bytes)"

class PythonFile(File):
    def __init__(self, name, size, is_executable=False):
        super().__init__(name, size)
        self.is_executable = is_executable

    def __repr__(self):
        return f"PythonFile(name={self.name}, size={self.size} bytes, executable={self.is_executable})"

class Directory:
    def __init__(self, name, parent=None):
        self.name = name
        self.parent = parent
        self.children = []
        self.files = []

    def add_file(self, file):
        self.files.append(file)

    def add_directory(self, directory_name):
        new_directory = Directory(directory_name, self)
        self.children.append(new_directory)
        return new_directory

    def remove_file(self, file_name):
        self.files = [file for file in self.files if file.name != file_name]

    def find_directory(self, path):
        if path == self.name:
            return self
        for child in self.children:
            result = child.find_directory(path)
            if result:
                return result
        return None

    def __repr__(self):
        return f"Directory(name={self.name})"

    def print_structure(self, indent=0):
        print(' ' * indent + self.name + '/')
        for file in self.files:
            print(' ' * (indent + 4) + file.__repr__())
        for child in self.children:
            child.print_structure(indent + 4)



class RootDirectory(Directory):
    def __init__(self, root_path=None, skip_hidden=True):
        if root_path:
            super().__init__(os.path.basename(root_path) or root_path)
            self.load_structure(root_path, skip_hidden)
        else:
            super().__init__('root')

    def load_structure(self, root_path, skip_hidden):
        if not os.path.isdir(root_path):
            return None

        for dirpath, dirnames, filenames in os.walk(root_path, topdown=True):
            current_dir = self.find_directory(os.path.basename(dirpath) or dirpath)
            if current_dir is None:
                continue

            if skip_hidden:
                dirnames[:] = [d for d in dirnames if not d.startswith('.')]

            for dirname in dirnames:
                current_dir.add_directory(dirname)

            for filename in filenames:
                if skip_hidden and filename.startswith('.'):
                    continue
                filepath = os.path.join(dirpath, filename)
                file_size = os.path.getsize(filepath)
                if filename.endswith('.py'):
                    current_dir.add_file(PythonFile(filename, file_size))
                else:
                    current_dir.add_file(File(filename, file_size))

    def save_to_file(self, file_path):
        with open(file_path, 'wb') as f:
            pickle.dump(self, f)

    @staticmethod
    def load_from_file(file_path):
        with open(file_path, 'rb') as f:
            return pickle.load(f)
        



In [49]:
# Example usage:
root_dir = RootDirectory(CODEBASE_ROOT)
if root_dir:
    root_dir.save_to_file('root_directory.pkl')

In [50]:
loaded_root_dir = RootDirectory.load_from_file('root_directory.pkl')
if loaded_root_dir:
    loaded_root_dir.print_structure()

oa-small-dev/
    File(name=LICENSE, size=35148 bytes)
    File(name=requirements.txt, size=1454 bytes)
    PythonFile(name=start_architecting.py, size=619 bytes, executable=False)
    File(name=README.md, size=329 bytes)
    src/
        PythonFile(name=language_models.py, size=142 bytes, executable=False)
        PythonFile(name=models.py, size=345 bytes, executable=False)
        agents/
            architect/
                PythonFile(name=__init__.py, size=19418 bytes, executable=False)
        helpers/
            PythonFile(name=github.py, size=1086 bytes, executable=False)


### Summarize pyfiles

In [71]:
import glob

# Define the glob pattern to find all document files
pattern = f"{CODEBASE_ROOT}/**/*.py"

# Use glob.glob with the pattern and recursive=True to search subdirectories
files = glob.glob(pattern, recursive=True)

# Print the list of document files found
py_files = []
for file in files:
    py_files.append(file)

In [83]:
import ast
import astor


for file in py_files:
    with open(file, "r") as f:
        source_code = f.read()
        tree = ast.parse(source_code)
        print(file, int((4/3)*len(astor.to_source(tree).split())))

/Users/macbook/Documents/GitHub/oa-small-dev/start_architecting.py 77
/Users/macbook/Documents/GitHub/oa-small-dev/src/language_models.py 13
/Users/macbook/Documents/GitHub/oa-small-dev/src/models.py 52
/Users/macbook/Documents/GitHub/oa-small-dev/src/agents/architect/__init__.py 2328
/Users/macbook/Documents/GitHub/oa-small-dev/src/helpers/github.py 114


In [66]:
tree.body

In [60]:
tree.body

for node in ast.walk(tree):
    print(astor.to_source(node))
    # if isinstance(node, ast.FunctionDef):
    #     print(node.name)
    #     print(astor.to_source(node))
    #     print()

import dspy
turbo = dspy.OpenAI(model='gpt-3.5-turbo-0125', max_tokens=4096)
gpt4 = dspy.OpenAI(model='gpt-4-0125-preview', max_tokens=4096)

import dspy

turbo = dspy.OpenAI(model='gpt-3.5-turbo-0125', max_tokens=4096)

gpt4 = dspy.OpenAI(model='gpt-4-0125-preview', max_tokens=4096)

dspy

turbo

dspy.OpenAI(model='gpt-3.5-turbo-0125', max_tokens=4096)

gpt4

dspy.OpenAI(model='gpt-4-0125-preview', max_tokens=4096)



AttributeError: No defined handler for node of type Store

In [ ]:
def get_function_code(tree):
    for node in ast.walk(tree):
        if isinstance(node, ast.FunctionDef):
            # Using astor to convert AST back to source code
            function_code = astor.to_source(node)
            return function_code.strip()
